In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14862842283405597820
xla_global_id: -1
]


In [2]:
from tensorflow.keras.datasets import boston_housing
(X_train, y_train), (X_test, y_test) = boston_housing.load_data()

In [4]:
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow import nn


class ChannelLayer(Model):
    def __init__(self):
        super().__init__()
        super(ChannelLayer, self).__init__()
        self.avg_pool = layers.AveragePooling2D(pool_size=(1,1))
        self.max_pool = layers.MaxPooling2D(pool_size=(1, 1))
        
        # self.fc = nn.Sequential(nn.Conv2d(in_p, in_p // red, 1, bias=False), nn.ReLU(), nn.Conv2d(in_p // red, in_planes, 1, bias=False))
        # self.sigmoid = nn.Sigmoid()
        
    def call(self, inputs):
        # avg_out = self.fc(self.avg_pool(x))
        # max_out = self.fc(self.max_pool(x))
        # out = avg_out + max_out
        # return self.sigmoid(out) * x
        return inputs

class SpatialLayer(Model):
    def __init__(self):
        super(SpatialLayer).__init__()
        # kernel_size = 7
        # self.compress = ChannelPool()
        self.conspatialv1 = layers.Conv2D(1, (3,3), padding='same') # BasicConv(2, 1, kernel_size, stride=1, padding=(kernel_size-1) // 2, relu=False)
        
    def call(self, inputs):
        # x_compress = self.compress(x)
        # x_out = self.spatial(x_compress)
        # scale = F.sigmoid(x_out) # broadcasting
        # return x * scale
        return inputs

class CBAM(Model):
    def __init__(self, gate_channels, spatial=True, channel=True):
        super().__init__()
        self.spatial = spatial
        self.channel = channel
        if self.channel:
            self.layer_channel = ChannelLayer()
        if self.spatial:
            self.layer_spatial = SpatialLayer()
    
    def call(self, inputs):
        if self.channel:
            inputs = self.layer_channel(inputs)
        if self.spatial:
            inputs = self.layer_spatial(inputs)
        return inputs

class Convolutional(Model):
    def __init__(self, input_size):
        """Falta ajustar los parmámetros de las capas convolucionales"""
        super(Convolutional).__init__()                                   
        self.conv1 = layers.Conv2D(1, (3,3), padding='same', activation='relu')   # nn.Conv2d(in_size, 6, 5)
        self.pool = layers.MaxPooling2D(pool_size=(2, 2))
        self.conv2 = layers.Conv2D(1, (3,3), padding='same', activation='relu')   # nn.Conv2d(6, 16, 5)
        
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.pool(x)
        return x

class MyModel(Model):
    def __init__(self, spatial=True, channel=True):
        super().__init__()
        
        # Elegimos si queremos que se ejecuten los mecanismos atencionales
        self.att = channel or spatial
        if self.att:
            self.layer_att = CBAM(spatial=spatial, channel=channel)
        self.layer_conv = Convolutional()
        self.layer_lineal = Lineal()
        
    def call(self, inputs):
        if self.att:
            inputs = self.layer_att(inputs)
        inputs = self.layer_conv(inputs)
        return self.layer_linal(inputs)
    
model = MyModel()

opt = optimizers.SGD(lr=0.001, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

model.build((32, 32, 3))
model.summary()

TypeError: __init__() missing 1 required positional argument: 'gate_channels'